# Example of using GSC Utils

In this demo, a set of pre-authorized credentials have been saved as .gsc-creds. Remember to be vigilant with permissions on this file. Use `chmod g-r .gsc-creds` to prevent users in your group from being able to read the file.

Install the package from GitHub with:

```
pip3 install -U git+https://github.com/bearloga/gsc-utils.git
```

In [1]:
import os
from gsc_utils import utils, sites, performance


You can find the source for Google Search Console Utilities at https://github.com/bearloga/gsc-utils



In [2]:
creds = utils.load_credentials(os.path.join('../cache', os.readlink('../cache/.gsc-creds')))

In [3]:
site_list = sites.list(creds)
site_list.head(2)

,siteUrl
6,https://commons.m.wikimedia.org/
5,https://commons.wikimedia.org/


This particular set of credentials has access to Google Search Console data for the two variants of Wikimedia Commons website – desktop and mobile web.

In [4]:
site_urls = ['commons.wikimedia.org', 'commons.m.wikimedia.org']
commons = performance.stats(creds, site_urls, '2019-12-31', '2020-01-01')

In [5]:
commons.columns

Index(['site', 'date', 'clicks', 'impressions', 'ctr', 'position'], dtype='object')

Since the returned stats are a Pandas DataFrame, we can relatively easily calculate the difference in average position between the two days for each variant:

In [6]:
(
    commons[['site', 'date', 'position']]
    .pivot(index='site', columns='date', values='position')
    .assign(difference=lambda df: df['2020-01-01']-df['2019-12-31'])
    .drop(columns = ['2020-01-01', '2019-12-31'])
)

date,difference
site,
https://commons.m.wikimedia.org/,0.086509
https://commons.wikimedia.org/,0.041977


On New Year's Day both variants appeared slightly lower in search results than they did on New Year's Eve, on average.

## Additional functionality

The function `fetch.stats()` has a `split_by` argument which can be used to get a breakdown of the stats broken down by one or two dimensions:

- `split_by=None` (default) just returns daily counts
- `split_by='country'` splits by ISO 3166-1 alpha-3 country code
- `split_by='device'` splits by device (Desktop, Tablet, Mobile)
- `split_by='country-device'` splits by device *and* ISO 3166-1 alpha-3 country code

By default, `fetch.stats()` has `use_https=True` because almost all of the properties in GSC are HTTPS and only few non-secure variants are still tracked. Set it to `False` when you need the HTTP variant (assuming it's registered in there).

For sites (properties) which have [structured data](https://developers.google.com/search/docs/guides/intro-structured-data), you can obtain statistics for rich card results only by setting `rich_results=True` (`False` by default). Be aware that:

> Dimensioning or filtering by search appearance will aggregate data by page rather than by property in the table

Refer to [official documentation](https://support.google.com/webmasters/answer/7576553?hl=en&ref_topic=9384513) for more information.